# ようこそ

maDMPへようこそ。ここでは、DMPに入力いただいた研究分野等情報に基づき、ワークフローを最適化します。  
以下のセルを実行することで、ワークフローを生成し、DMP情報を基に最適化します。

In [ ]:
# DMP情報
field = '%v'

In [ ]:
%%%sh
# ワークフローテンプレート取得
# 検証のため、参照先をivis-kuwata/workflow-templateの
# issue/merge-templateブランチに変更中
git clone -b issue/merge-template https://github.com/ivis-kuwata/workflow-template.git /home/jovyan/WORKFLOW
rm -rf /home/jovyan/WORKFLOW/.git

In [ ]:
# dmp.jsonに"fields"プロパティがある想定
import os
import glob
import shutil

path_flows = os.path.join('WORKFLOW', 'FLOW')

templates = glob.glob(os.path.join(path_flows, '**'), recursive=True)

# 選択外の分野のセクション群を削除
for tmpl in templates:
    file = os.path.basename(tmpl)
    if not os.path.isdir(tmpl) and os.path.splitext(file)[1] == '.ipynb':
        if 'base_' not in file and field not in file:
            os.remove(tmpl)
            
print('最適化処理が完了しました。次の手順へお進みください。')

# ワークフロー全体の関連図と目次を生成する

次に、最適化したワークフロー全体が俯瞰できる図を生成します。  
以下を実行することで、システムがワークフロー関連図、および目次を作成します。

◆◆◆開発メモ◆◆◆

*.diagに分野固有のセクションはまだ反映されていない？

In [ ]:
%%%!
# 必要パッケージのインストール
pip install lxml
pip install blockdiag

# 日本語対応フォントのインストール
mkdir ~/.fonts
cd ~/.fonts
wget http://archive.ubuntu.com/ubuntu/pool/universe/f/fonts-ipafont/fonts-ipafont-gothic_00303-18ubuntu1_all.deb
dpkg-deb -x fonts-ipafont-gothic_00303-18ubuntu1_all.deb ~/.fonts
cp ~/.fonts/usr/share/fonts/opentype/ipafont-gothic/ipag.ttf ~/.fonts/ipag.ttf

In [ ]:
from IPython.display import clear_output

# 関連図を作成する
%%run /home/jovyan/WORKFLOW/scripts/nb_utils.py
setup_diag()
filename=generate_svg_diag(diag='/home/jovyan/WORKFLOW/images/notebooks.diag')

clear_output()
print('関連図の作成が完了しました。')


In [ ]:
from IPython.display import Markdown
from IPython.display import clear_output

%%cd /home/jovyan/WORKFLOW
%%run scripts/nb_utils.py
%%cd FLOW
md_list = [
    '## フロー支援機能の使い方',
    '「ワークフロー実行準備」を最初に実行したあと、研究準備フェーズから実験期フェーズ、実験終了後フェーズの順に進んでください。<br>各フェーズでは、リンクを押下して、表示される画面の手順に沿って実行することでワークフローを実施できます。  ',
    '以下で流れを確認してから、ワークフロー実施に移ってください。'
    '[![関連図](../images/notebooks.svg)](../images/notebooks.svg)',
    '## 0. ワークフローを実行するための準備をする',
    '次のフェーズに進む前に必ず実行する必要があります。<br>ワークフロー機能を使うのが初めての場合は、「FLOWの立ち上げ毎に実行」、「初回立ち上げ時のみ実行」の順にで実行してください。<br>2回目以降は、ワークフロー機能を実行する前に「FLOWの立ち上げ毎に実行」を実行してください。',
    str(notebooks_toc('util')),
    '## 1. 研究準備フェーズ',
    '研究準備フェーズでは実験前の準備をサポートします。',
    str(notebooks_toc('01_preparation_phase')),
    '## 2. 実験期フェーズ',
    '実験期フェーズでは実験の開始から、複数の実験の再現性の確認などをサポートします。<br>まずは、「実験を開始する」から実験を作成してください。実験の終了後、複数の実験の再現性を確認したいタイミングで「実験パイプラインをチェックする」「実験パイプラインを実行する」を実行してください。',
    str(notebooks_toc('02_experimental_phase')),
    '## 3 実験終了後フェーズ',
    '実験終了後フェーズでは研究成果の公開をサポートします。',
    str(notebooks_toc('03_after_research_phase'))
]
%%cd ~/

# 目次をFLOW.mdとして書き出す
try:
    with open('WORKFLOW/FLOW/FLOW.md', 'w') as f:
        f.write('\n'.join(md_list))
        clear_output()
        print('関連図・目次が作成されました。以下のリンクをクリックしてご覧いただけます。')
        print('https://jupyter.cs.rcos.nii.ac.jp' + os.environ['JUPYTERHUB_SERVICE_PREFIX'] + 'notebooks/WORKFLOW/FLOW/FLOW.md')
except:
    clear_output()
    print('申し訳ありません。エラーが発生しました。')